In [1]:
!pip install accelerate \
  bitsandbytes \
  peft \
  transformers \
  sentencepiece \
    scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 34.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 85.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 66.9 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.9/773.9 kB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 126.8 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Fri Nov 24 12:50:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          On   | 00000000:23:00.0 Off |                    0 |
|  0%   25C    P8    17W / 300W |      0MiB / 46068MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

In [2]:
MODEL_NAME = "IlyaGusev/saiga2_13b_lora"
BASE_MODEL_PATH = "TheBloke/Llama-2-13B-fp16"
# BASE_MODEL_PATH = "meta-llama/Llama-2-13b-hf"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

In [3]:
class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        start_token_id=1,
        bot_token_id=9225
    ):
        self.message_template = message_template
        self.start_token_id = start_token_id
        self.bot_token_id = bot_token_id
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def get_start_token_id(self):
        return self.start_token_id

    def get_bot_token_id(self):
        return self.bot_token_id

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += tokenizer.decode([self.start_token_id, self.bot_token_id])
        return final_text.strip()

In [4]:
def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_PATH,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 5120, padding_idx=0)
        (layers): ModuleList(
          (0-39): 40 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear8bitLt(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (base_layer): Linear8bitLt(in_features=5120, out_features=5120, bias=False)
              )
              (k_proj): Linear8bitLt(
                (lora_dropout)

In [8]:
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.15,
  "temperature": 0.2,
  "top_k": 30,
  "top_p": 0.9
}



In [9]:
def model_answer(input):
    conversation = Conversation()
    conversation.add_user_message(input)
    prompt = conversation.get_prompt(tokenizer)
    model_output = generate(model, tokenizer, prompt, generation_config)
    torch.cuda.empty_cache()
    return model_output

In [10]:
import pandas as pd
from collections import Counter
import random

In [11]:
df = pd.read_csv("train_dataset_train_variant2.csv")

In [12]:
df

,Исполнитель,Группа тем,Текст инцидента,Тема
0,Лысьвенский городской округ,Благоустройство,"Добрый день. Сегодня, 20.08.22, моя мать шла ...",★ Ямы во дворах
1,Министерство социального развития ПК,Социальное обслуживание и защита,"Пермь г, 79194692145. В Перми с ноября 2021 г...",Оказание гос. соц. помощи
2,Министерство социального развития ПК,Социальное обслуживание и защита,Добрый день . Скажите пожалуйста если подовал...,Дети и многодетные семьи
3,Город Пермь,Общественный транспорт,Каждая из них не о чем. Люди на остановках хо...,Содержание остановок
4,Министерство здравоохранения,Здравоохранение/Медицина,В Березниках у сына привитого откоронавируса ...,Технические проблемы с записью на прием к врачу
...,...,...,...,...
22525,Министерство социального развития ПК,Социальное обслуживание и защита,", а если ещё не погасили ипотеку, но площадь ...",Улучшение жилищных условий
22526,Губахинский городской округ,ЖКХ,Город Гремячинск ситуация с теплом на улице Л...,Ненадлежащее качество или отсутствие отопления
22527,Министерство здравоохранения,Здравоохранение/Медицина,"Здравствуйте у меня ребёнку 2 месяца , тест н...",Технические проблемы с записью на прием к врачу
22528,Лысьвенский городской округ,Благоустройство,А что творится с благоустройством дворов. Воо...,Благоустройство придомовых территорий


In [13]:
group_counter = Counter(df['Группа тем'])
group_counter.most_common()

[('Здравоохранение/Медицина', 4593),
 ('Социальное обслуживание и защита', 4506),
 ('Дороги', 3044),
 ('ЖКХ', 2637),
 ('Благоустройство', 2441),
 ('Мусор/Свалки/ТКО', 1673),
 ('Общественный транспорт', 850),
 ('Коронавирус', 801),
 ('Образование', 689),
 ('Безопасность', 286),
 ('Связь и телевидение', 222),
 ('Мобилизация', 129),
 ('Физическая культура и спорт', 118),
 ('Строительство и архитектура', 107),
 ('Газ и топливо', 80),
 ('Спецпроекты', 80),
 ('Культура', 68),
 ('Электроснабжение', 37),
 ('Экономика и бизнес', 28),
 ('Экология', 25),
 ('Роспотребнадзор', 23),
 ('Памятники и объекты культурного наследия', 22),
 ('Государственная собственность', 22),
 ('Торговля', 20),
 ('МФЦ "Мои документы"', 18),
 ('Погребение и похоронное дело', 11)]

In [14]:
with open('classification_report.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    print(text)

                                          precision    recall  f1-score   support

                            Безопасность       0.80      0.51      0.62        94
                         Благоустройство       0.57      0.58      0.57       806
                           Газ и топливо       0.75      0.23      0.35        26
           Государственная собственность       0.00      0.00      0.00         7
                                  Дороги       0.69      0.76      0.72      1005
                                     ЖКХ       0.74      0.77      0.76       870
                Здравоохранение/Медицина       0.81      0.89      0.85      1516
                             Коронавирус       0.66      0.38      0.48       264
                                Культура       0.38      0.13      0.19        23
                     МФЦ "Мои документы"       0.00      0.00      0.00         6
                             Мобилизация       0.75      0.35      0.48        43
               

In [16]:
def create_prompt(group, theme, examples, permormer=None,):
    prompt = f"""Я хочу создать реалистичные обращения пользователей социальных сетей по следующей теме: {theme}. Ниже представлены образцы реальных обращений:\n{examples}\nПожалуйста, cгенерируй строго пять подобных обращений, сохраняя соответствующий стиль, тональность и формат, указанный в примерах. Каждое обращение должно быть уникальным и отражать возможные вопросы, проблемы или мнения пользователей по этой теме в социальных сетях.\nОтвет:"""
    return prompt

In [21]:
from tqdm.notebook import tqdm
import time
import re

In [22]:
# groups = ['Погребение и похоронное дело', 'МФЦ "Мои документы"', 'Торговля', 'Государственная собственность',
#           'Памятники и объекты культурного наследия', 'Роспотребнадзор', 'Экология',  'Экономика и бизнес', 'Электроснабжение',
#           'Культура', 'Спецпроекты', 'Газ и топливо']
groups = ['МФЦ "Мои документы"', 'Торговля', 'Государственная собственность',
          'Памятники и объекты культурного наследия', 'Роспотребнадзор', 'Экология',  'Экономика и бизнес', 'Электроснабжение',
          'Культура', 'Спецпроекты', 'Газ и топливо']
df_res = pd.DataFrame(columns=['Текст инцидента', 'Группа тем', 'Тема', 'Исполнитель'])
for g in groups:
    print(g)
    count = 0
    while count < 100:
        df_g = df[df['Группа тем'] == g]
        themes = list(df_g['Тема'].unique())
        #print(themes)
        for t in themes:
            df_t = df_g[df_g['Тема'] == t]
            #print(df_t)
            performers = list(df_t['Исполнитель'].unique())
            #print(performers)
            for p in performers:
                try:
                    start_time = time.time()
                    #print(p)
                    df_p = df_t[df_t['Исполнитель'] == p]
                    texts = df_p['Текст инцидента'].tolist()
                    if len(texts) < 5:
                        l = len(texts)
                    else:
                        l = 5
                    random_text_samples = random.sample(texts, l)
                    examples_string = "\n".join([f"{ind + 1}. {t}" for ind, t in enumerate(random_text_samples)])
                    #print(examples_string)
                    model_prompt = create_prompt(group=g, theme=t, examples=examples_string)
                    #print(model_prompt)
                    #print('=================================')
                    model_output = model_answer(model_prompt)
                    #print(model_output)
                    ex = [re.sub(r"^\d+.\s", "", el) for el in model_output.split('\n')]
                    #print(ex)
                    res = []
                    for el in ex:
                        res.append({'Текст инцидента': el, 'Группа тем': g, 'Тема': t, 'Исполнитель': p})
                    end_time = time.time() - start_time
                    print(f'Сгенерено: {len(res)} | Всего: {count} | Время: {end_time}')
                    df_to_add = pd.DataFrame(res)
                    df_res = pd.concat([df_res, df_to_add], ignore_index=True, sort=False)
                    df_res.to_csv('res_df.csv', index=False)
                    count += len(res)
                except Exception as e:
                    error_name = type(e).name
                    print(f"Caught an error: {error_name}")
                    print(f"EXCEPTION: {e}")
                    print(sys.exc_info()[0])
                    traceback.print_exc(file=sys.stdout)
                    pass
    

МФЦ "Мои документы"
Сгенерено: 8 | Всего: 0 | Время: 107.39320015907288
Сгенерено: 4 | Всего: 8 | Время: 44.569766998291016
Сгенерено: 5 | Всего: 12 | Время: 66.01557469367981
Сгенерено: 5 | Всего: 17 | Время: 51.004541873931885
Сгенерено: 4 | Всего: 22 | Время: 49.22175431251526
Сгенерено: 7 | Всего: 26 | Время: 81.70200896263123
Сгенерено: 5 | Всего: 33 | Время: 61.85924291610718
Сгенерено: 5 | Всего: 38 | Время: 55.60674715042114
Сгенерено: 5 | Всего: 43 | Время: 44.49463200569153
Сгенерено: 4 | Всего: 48 | Время: 44.35603451728821
Сгенерено: 5 | Всего: 52 | Время: 66.25730204582214
Сгенерено: 5 | Всего: 57 | Время: 73.05565977096558
Сгенерено: 4 | Всего: 62 | Время: 50.95707702636719
Сгенерено: 4 | Всего: 66 | Время: 42.76314425468445
Сгенерено: 5 | Всего: 70 | Время: 35.715678453445435
Сгенерено: 9 | Всего: 75 | Время: 151.48759603500366
Сгенерено: 5 | Всего: 84 | Время: 77.61836123466492
Сгенерено: 4 | Всего: 89 | Время: 44.78032326698303
Сгенерено: 8 | Всего: 93 | Время: 80.0248

In [39]:
pd.set_option('display.max_rows', df_res.shape[0]+1)
df_res.iloc[1201:1247]

,Текст инцидента,Группа тем,Тема,Исполнитель
1201,,Газ и топливо,Запрос на газификацию и её условия,Губахинский городской округ
1202,"Вопросы, которые мы будем обсуждать:",Газ и топливо,Запрос на газификацию и её условия,Губахинский городской округ
1203,- Какие преимущества газификации для города?,Газ и топливо,Запрос на газификацию и её условия,Губахинский городской округ
1204,- Как быстро можно провести газовое освещение ...,Газ и топливо,Запрос на газификацию и её условия,Губахинский городской округ
1205,- Стоит ли ждать государственной поддержки для...,Газ и топливо,Запрос на газификацию и её условия,Губахинский городской округ
1206,- Какой уровень газа в воздухе безопасен для з...,Газ и топливо,Запрос на газификацию и её условия,Губахинский городской округ
1207,- Как часто нужно проводить техническое обслуж...,Газ и топливо,Запрос на газификацию и её условия,Губахинский городской округ
1208,,Газ и топливо,Запрос на газификацию и её условия,Губахинский городской округ
1209,Мы готовы ответить на все ваши вопросы и расск...,Газ и топливо,Запрос на газификацию и её условия,Губахинский городской округ
1210,,Газ и топливо,Запрос на газификацию и её условия,Губахинский городской округ


In [43]:
df = df_res.drop(list(range(1201,1247)))

In [44]:
df

,Текст инцидента,Группа тем,Тема,Исполнитель
0,Здравствуйте! Недавно я пытался зарегистрирова...,"МФЦ ""Мои документы""",Государственные услуги,Министерство здравоохранения
1,,"МФЦ ""Мои документы""",Государственные услуги,Министерство здравоохранения
2,Добрый день! Я недавно подавал заявление на го...,"МФЦ ""Мои документы""",Государственные услуги,Министерство здравоохранения
3,,"МФЦ ""Мои документы""",Государственные услуги,Министерство здравоохранения
4,"Привет! Я недавно получил статус ""постоянного ...","МФЦ ""Мои документы""",Государственные услуги,Министерство здравоохранения
5,,"МФЦ ""Мои документы""",Государственные услуги,Министерство здравоохранения
6,Здравствуйте! Я недавно подал заявление на пол...,"МФЦ ""Мои документы""",Государственные услуги,Министерство здравоохранения
7,Здравствуйте! Я недавно начал работать на нову...,"МФЦ ""Мои документы""",Государственные услуги,Министерство здравоохранения
8,Здравствуйте! Я заказал государственную услугу...,"МФЦ ""Мои документы""",Государственные услуги,Город Пермь
9,"Привет! Хотел бы получить информацию о том, ка...","МФЦ ""Мои документы""",Государственные услуги,Город Пермь


In [45]:
df_f = pd.read_csv('first_theme.csv')

In [46]:
super_res_df = pd.concat([df_f, df], ignore_index=True, sort=False)

In [47]:
super_res_df

,Текст инцидента,Группа тем,Тема,Исполнитель
0,Погребения - это серьезная проблема в нашей ст...,Погребение и похоронное дело,Погребение и похоронное дело,Город Пермь
1,"Я также считаю, что мы должны делать все возмо...",Погребение и похоронное дело,Погребение и похоронное дело,Город Пермь
2,"Я думаю, что многие люди уже готовы переходить...",Погребение и похоронное дело,Погребение и похоронное дело,Город Пермь
3,"Я считаю, что строительство новых крематориев ...",Погребение и похоронное дело,Погребение и похоронное дело,Город Пермь
4,"Я думаю, что мы должны обсуждать этот вопрос о...",Погребение и похоронное дело,Погребение и похоронное дело,Город Пермь
...,...,...,...,...
1331,Здравствуйте! Я планирую провести газификацию ...,Газ и топливо,"Стоимость, оплата и возврат средств на газифик...",Министерство социального развития ПК
1332,Привет! Я заинтересован в получении субсидии н...,Газ и топливо,"Стоимость, оплата и возврат средств на газифик...",Город Пермь
1333,"Добрый день! Я прочитал, что есть программа фи...",Газ и топливо,"Стоимость, оплата и возврат средств на газифик...",Город Пермь
1334,"Привет! Хотел бы узнать, есть ли какая-то прог...",Газ и топливо,"Стоимость, оплата и возврат средств на газифик...",Город Пермь


In [48]:
super_res_df.dropna()

,Текст инцидента,Группа тем,Тема,Исполнитель
0,Погребения - это серьезная проблема в нашей ст...,Погребение и похоронное дело,Погребение и похоронное дело,Город Пермь
1,"Я также считаю, что мы должны делать все возмо...",Погребение и похоронное дело,Погребение и похоронное дело,Город Пермь
2,"Я думаю, что многие люди уже готовы переходить...",Погребение и похоронное дело,Погребение и похоронное дело,Город Пермь
3,"Я считаю, что строительство новых крематориев ...",Погребение и похоронное дело,Погребение и похоронное дело,Город Пермь
4,"Я думаю, что мы должны обсуждать этот вопрос о...",Погребение и похоронное дело,Погребение и похоронное дело,Город Пермь
...,...,...,...,...
1331,Здравствуйте! Я планирую провести газификацию ...,Газ и топливо,"Стоимость, оплата и возврат средств на газифик...",Министерство социального развития ПК
1332,Привет! Я заинтересован в получении субсидии н...,Газ и топливо,"Стоимость, оплата и возврат средств на газифик...",Город Пермь
1333,"Добрый день! Я прочитал, что есть программа фи...",Газ и топливо,"Стоимость, оплата и возврат средств на газифик...",Город Пермь
1334,"Привет! Хотел бы узнать, есть ли какая-то прог...",Газ и топливо,"Стоимость, оплата и возврат средств на газифик...",Город Пермь


In [56]:
clean_df = super_res_df[super_res_df['Текст инцидента'].notna()]

In [73]:
clean_df = clean_df[clean_df['Текст инцидента'] != '']

In [75]:
clean_df = clean_df[clean_df['Текст инцидента'] != 'Ответ:']

In [77]:
clean_df = clean_df[['Исполнитель', 'Группа тем', 'Текст инцидента', 'Тема']]

In [78]:
clean_df.to_csv('generated_train_saiga_v1.csv', index=False)

In [79]:
clean_df.duplicated(keep='first').sum()

1